In [2]:
%cd /app

/app


In [8]:
import os
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoFeatureExtractor
from src.config import CFG
from src.data_loader import create_dataloader
from src.model import load_model
from src.train import train  # Importing train function from train.py


def main():
    # Set paths
    BASE_PATH = "/app/birdclef-2024"
    master_path = os.path.join(BASE_PATH, "master.csv")
    csv_file = os.path.join(BASE_PATH, "filtered_data_with_labels.csv")

    # Load the feature extractor for AST model
    print("Loading feature extractor...")
    extractor = AutoFeatureExtractor.from_pretrained(CFG.feature_extractor_name)

    # Load and prepare filtered data
    print("Loading filtered data...")
    df = pd.read_csv(csv_file).drop(columns='y')
    df["filename"] = df["filename"].str.split('/').apply(lambda x: '/'.join(x[-2:]))
    df.index = df["filename"]

    # Balance the number of samples for each class
    df_sampled = df.groupby('primary_label').sample(n=df.primary_label.value_counts().min(), random_state=42)

    # Split into train and test datasets
    print("Splitting data into train and test sets...")
    df_train, df_test = train_test_split(df_sampled, test_size=0.2, stratify=df_sampled['primary_label'], random_state=42)

    # Load and prepare master data
    print("Loading master data...")
    master = pd.read_csv(master_path).iloc[:, 1:]
    master.index = master["filename"]

    # Align train and test sets with master data using their index
    df_train = master.loc[df_train.index].reset_index(drop=True)
    df_test = master.loc[df_test.index].reset_index(drop=True)

    classes = df_train['primary_label'].unique()
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=df_train['primary_label'])
    
    # Compute num_labels dynamically based on the unique values in 'primary_label'
    num_labels = len(df_train['primary_label'].unique())
    print(f"Number of unique labels: {num_labels}")

    # Load the model with the correct num_labels
    print("Loading model...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = load_model(CFG.model_name, num_labels=num_labels, device=device)

    # Move model to GPU and convert to float16
    model = model.to(device)  # Convert model to float16
    print(f"Model is running on: {device}")
    
    # Check train and test sizes and unique labels
    print(f"Train set size: {len(df_train)} samples")
    print(f"Test set size: {len(df_test)} samples")
    print(f"Unique labels in train set: {len(df_train['primary_label'].unique())}")
    print(f"Unique labels in test set: {len(df_test['primary_label'].unique())}")

    # Create DataLoader for training
    print("Creating DataLoader for training data...")
    train_loader = create_dataloader(csv_file=csv_file, base_path=BASE_PATH, batch_size=CFG.batch_size, extractor=extractor)

    # Create DataLoader for testing
    print("Creating DataLoader for testing data...")
    test_loader = create_dataloader(csv_file=csv_file, base_path=BASE_PATH, batch_size=CFG.batch_size, extractor=extractor, shuffle=False, test=True)


    # Convert class weights to a tensor
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device) # Ensure it's on the correct device
    
    # Start training by calling train function from train.py
    train(model, train_loader, test_loader, device, CFG, class_weights_tensor)


In [9]:
main()

Loading feature extractor...
Loading filtered data...
Splitting data into train and test sets...
Loading master data...
Number of unique labels: 89
Loading model...
Loading model with num_labels = 89
Original classifier structure: ASTMLPHead(
  (layernorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dense): Linear(in_features=768, out_features=527, bias=True)
)
Classifier input features: 768
Model is running on: cuda
Train set size: 1922 samples
Test set size: 481 samples
Unique labels in train set: 89
Unique labels in test set: 89
Creating DataLoader for training data...
Creating DataLoader for testing data...


Epoch 1/30: 100%|██████████| 481/481 [03:51<00:00,  2.07batch/s]


Epoch 1/30 - Training Loss: 4.5019


Evaluating: 100%|██████████| 121/121 [00:16<00:00,  7.54batch/s, accuracy=0.0229, f1_macro=0.0127, f1_weighted=0.02]    


Epoch 1/30
Validation Loss: 4.4529
Validation Accuracy: 0.0229
Validation Macro F1: 0.0127
Validation Weighted F1: 0.0200
New best model found at epoch 1. Saving model...
Model saved to ./saved_models/best_model_epoch_1.pth


...

Epoch 29/30
Validation Loss: 3.6975
Validation Accuracy: 0.3410
Validation Macro F1: 0.2346
Validation Weighted F1: 0.3340


Epoch 30/30: 100%|██████████| 481/481 [03:48<00:00,  2.10batch/s]


Epoch 30/30 - Training Loss: 3.3811


Evaluating: 100%|██████████| 121/121 [00:15<00:00,  7.76batch/s, accuracy=0.362, f1_macro=0.243, f1_weighted=0.353]

Epoch 30/30
Validation Loss: 3.6889
Validation Accuracy: 0.3617
Validation Macro F1: 0.2432
Validation Weighted F1: 0.3533
